In [37]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

In [38]:

import gensim, pickle
from gensim.models.keyedvectors import KeyedVectors
LdaModel = gensim.models.ldamodel.LdaModel


In [39]:
#ruta_word_embedding = '../data/wiki.multi.en.vec'
#ruta_word_embedding = '../data/embedding_english__europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.bin'
ruta_word_embedding = '../data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
wordembedding = gensim.models.Word2Vec.load(ruta_word_embedding)
#wordembedding = KeyedVectors.load_word2vec_format(ruta_word_embedding, binary=False)
#wordembedding = KeyedVectors.load_word2vec_format(ruta_word_embedding, binary=True)

In [40]:
'deletefacebook' in wordembedding.wv

True

In [41]:
len(wordembedding.wv.vocab)

23994

In [42]:
"""
Note: 
LDAvis Scenario:
If you wish to calculate topic similarity metric over one topic modeling output, therefore lda_model_collection_1
and lda_model_collection_2 should be the same. 

Sankey diagram scenario:
If you wish to calculate topic similarity metric over two different topic modeling outputs, lda_model_collection_1
and lda_model_collection_2 should be different


The same shouuld be done with the most relevant documents.
"""



lda_model_collecion_1 = LdaModel.load("../data/cambridge_analytica/regional_datasets/files_europe/english_europe_tweets_20190411.csv_gensim.model")
lda_model_collecion_2 = LdaModel.load("../data/cambridge_analytica/regional_datasets/files_europe/english_europe_tweets_20190411.csv_gensim.model")
#lda_model_collecion_2 = LdaModel.load("../data/cambridge_analytica/regional_datasets/files_northamerica/english_northamerica_tweets_20190411.csv_gensim.model")

In [43]:
with open('../data/cambridge_analytica/regional_datasets/files_europe/english_europe_tweets_20190411.csvsent_topics_sorteddf_mallet_ldamodel', 'rb') as f:
    most_relevant_documents_collection_1 = pickle.load(f)
most_relevant_documents_collection_1 = most_relevant_documents_collection_1[['Topic_Num','Topic_Perc_Contrib','text']]

# ../data/cambridge_analytica/regional_datasets/files_northamerica/english_northamerica_tweets_20190411.csvsent_topics_sorteddf_mallet_ldamodel
with open('../data/cambridge_analytica/regional_datasets/files_europe/english_europe_tweets_20190411.csvsent_topics_sorteddf_mallet_ldamodel', 'rb') as f:
    most_relevant_documents_collection_2 = pickle.load(f)
most_relevant_documents_collection_2 = most_relevant_documents_collection_2[['Topic_Num','Topic_Perc_Contrib','text']]


In [44]:
'''My own tokenizer '''
import unidecode
from string import punctuation
from string import digits
punctuation+="¡¿<>'`"
punctuation+='"'

#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 


def text_cleaner(tweet):
    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()
    tweet = unidecode.unidecode(tweet)
    #tweet = tweet.strip().split()
    #filtered_words = [word for word in tweet if word not in stopWords]
    #corpus[id_tweet]= filtered_words
    #id_tweet+=1
    tweet = tweet.split()
    return tweet


In [45]:
def get_dicts_relevant_keywords_documents(lda_model,df_relevant_documents, n_terms):
    num_topics = lda_model.num_topics
    #create dictionary of top keywords 
    topKeywordsDict = {}
    for topic_id in range(num_topics):
        topKeywordsDict[topic_id] = []
        for term, probability in lda_model.show_topic(topic_id,topn=n_terms):
            topKeywordsDict[topic_id].append({
                "term":term,
                "probability":probability
            })

    #create dictionary of relevant documents
    relevantDocumentsDict = {}
    for index,row in df_relevant_documents.iterrows():
        topic_id = int(row['Topic_Num'])
        if topic_id not in relevantDocumentsDict:
            relevantDocumentsDict[topic_id]=[]
        relevantDocumentsDict[topic_id].append({
            'topic_perc_contrib':row['Topic_Perc_Contrib'],
            'text':row['text']
        })
    return (topKeywordsDict, relevantDocumentsDict)

def getDocumentVector(text, wordembedding):
    #preprocesar    
    #encontrar palabras en word embedding
    #ponderas palabras TF-IDF
    document_vector = 0.0
    words_found = 0.0
    for word in text_cleaner(text):
        if word in wordembedding.wv:
            document_vector+=wordembedding.wv[word] #aqui hay que ponderar
            words_found+=1
    return document_vector/words_found

        
    
def get_topkeywords_relevantdocuments_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms): #n_terms : numero de top keywords a considerar
    topKeywordsDict, relevantDocumentsDict = get_dicts_relevant_keywords_documents(lda_model, most_relevant_documents, n_terms)

    ### Create top keyword vector per topic
    #create keyword vector
    topkeywords_vectors_dict = {}
    num_topics = lda_model.num_topics
    for topic_id in range(num_topics):
        topkeywords_vector = 0
        ranking = 1.0
        for item in topKeywordsDict[topic_id]:
            if item['term'] in wordembedding.wv: #no todas las palabras aparecerán en el ranking, que hacer con el resto
                #print(item['term'], item['probability'])
                topkeywords_vector += wordembedding.wv[item['term']]/ranking
            else:
                print(item['term']," position:",ranking)
            ranking+=1
        topkeywords_vectors_dict[topic_id] = topkeywords_vector
        
    relevantdocuments_vectors_dict = {}
    for topic_id in range(num_topics):
        relevantDocumentsvector = 0.0
        for item in relevantDocumentsDict[topic_id]:
            #quizas esto hacerlo para los primero 100 docs, 500 docs, el resto es un % pequeño que se pierde, optimizar
            #revisar si multiplicar por la contribucion es lo correcto, eso no da 1 o si? OHHHHHH, habria que sacar todooos los docs no solo los 100 primeros
            relevantDocumentsvector+= float(item['topic_perc_contrib'])*getDocumentVector(item['text'], wordembedding)
        relevantdocuments_vectors_dict[topic_id] = relevantDocumentsvector
        
    return (topkeywords_vectors_dict, relevantdocuments_vectors_dict)


In [46]:
def get_topic_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms, lambda_):
    num_topics = lda_model.num_topics
    topkeywords_vectors_dict, relevantdocuments_vectors_dict = get_topkeywords_relevantdocuments_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms)
    final_topic_vectors_dict = dict()
    for topic_id in range(num_topics):
        final_topic_vector = lambda_*topkeywords_vectors_dict[topic_id]+(1-lambda_)*relevantdocuments_vectors_dict[topic_id]
        final_topic_vectors_dict[topic_id] = final_topic_vector
    return final_topic_vectors_dict

In [47]:
def get_matrix(wordembedding, lda_model_1,most_relevant_documents_1,lda_model_2,most_relevant_documents_2, n_terms, lambda_):
    
    final_topic_vectors_dict_1 =  get_topic_vectors(wordembedding, lda_model_1,most_relevant_documents_1,  n_terms, lambda_)
    final_topic_vectors_dict_2 =  get_topic_vectors(wordembedding, lda_model_2,most_relevant_documents_2,  n_terms, lambda_)
    
    topic_similarity_matrix = []
    for i in range(lda_model_1.num_topics):
        row = []
        for j in range(lda_model_2.num_topics):
            topic_i = final_topic_vectors_dict_1[i].reshape(1,-1)
            topic_j = final_topic_vectors_dict_2[j].reshape(1,-1)
            row.append(float(cosine_similarity(topic_i,topic_j)))
        topic_similarity_matrix.append(row)
    topic_similarity_matrix= np.asarray(topic_similarity_matrix)
    return topic_similarity_matrix

In [48]:
topn = 30
#lambda_ = 0.8


i = 0.0
matrices_dict = dict()
while i <=1.01:
    lambda_ = round(i,2)
    print(lambda_)
    matrix = get_matrix(wordembedding, lda_model_collecion_1, most_relevant_documents_collection_1, lda_model_collecion_2, most_relevant_documents_collection_2,topn, lambda_)
    matrices_dict[lambda_] = matrix
    i+=0.01
 

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49
0.5
0.51
0.52
0.53
0.54
0.55
0.56
0.57
0.58
0.59
0.6
0.61
0.62
0.63
0.64
0.65
0.66
0.67
0.68
0.69
0.7
0.71
0.72
0.73
0.74
0.75
0.76
0.77
0.78
0.79
0.8
0.81
0.82
0.83
0.84
0.85
0.86
0.87
0.88
0.89
0.9
0.91
0.92
0.93
0.94
0.95
0.96
0.97
0.98
0.99
1.0


In [49]:
with open('../data/cambridge_analytica/regional_datasets/matrix_europe_vs_europe_own_wordembedding', 'wb') as f:
            pickle.dump(matrices_dict, f)

# OLD

In [33]:
#matrices_dict

In [34]:
"""
with open('../data/cambridge_analytica/matrix_collection_1_1', 'wb') as f:
            pickle.dump(matrices_dict, f)
"""
"""
with open('../data/cambridge_analytica/regional_datasets/files_europe/matrix_europe_vs_europe', 'wb') as f:
            pickle.dump(matrices_dict, f)
"""     



"\nwith open('../data/cambridge_analytica/regional_datasets/files_europe/matrix_europe_vs_europe', 'wb') as f:\n            pickle.dump(matrices_dict, f)\n"

In [35]:
"""



with open('../data/cambridge_analytica/matrix_collection_1_2', 'rb') as f:
    matrix_sankey_2 = pickle.load(f)
"""

"\n\n\n\nwith open('../data/cambridge_analytica/matrix_collection_1_2', 'rb') as f:\n    matrix_sankey_2 = pickle.load(f)\n"

In [36]:
#matrix_sankey_2[0.8]